# Money Laundering Patterns Preprocessing
This notebook transforms the data from HI-Small_Patterns.txt into a structured tabular format, generating two output files: HI-Small_Sampled_Laundering_Patterns.csv and HI-Small_Sampled_5491.csv. These files are tailored for use in the GFP XGBoost Pipeline.

In [8]:
import pandas as pd
import regex as re
import random

In [15]:
def parse_text(text):
    all_rows = []
    laundering_grp_rows = []
    laundering_pattern = ""
    laundering_details = ""
    current_group = 0

    for line in text.split("\n"):
        line = line.strip()
        if line.startswith("BEGIN LAUNDERING ATTEMPT - "):
            current_group += 1
            laundering_patterns = re.search(
                r"BEGIN LAUNDERING ATTEMPT - (.+?):", line
            )
            if laundering_patterns:
                laundering_pattern = laundering_patterns.group(1).strip()

            laundering_details = re.search(
                r"BEGIN LAUNDERING ATTEMPT - (.+?):(.+)", line
            )
            if laundering_details:
                laundering_details = laundering_details.group(2).strip()
            laundering_grp_rows.append([laundering_pattern, laundering_details, current_group])
        elif line.startswith("END LAUNDERING ATTEMPT - "):
            continue
        elif line:
            parts = line.split(",")
            all_rows.append(
                parts + [laundering_pattern, laundering_details, current_group]
            )

    return laundering_grp_rows, all_rows


def convert_to_dataframe(laundering_grp_rows, all_rows):
    # Convert to DataFrame
    df_1 = pd.DataFrame(
        laundering_grp_rows,
        columns=["Pattern", "Pattern Details", "Same Group"],
    )

    df_2 = pd.DataFrame(
        all_rows,
        columns=[
            "Timestamp",
            "From Bank",
            "From Account",
            "To Bank",
            "To Account",
            "Amount Received",
            "Receiving Currency",
            "Amount Paid",
            "Payment Currency",
            "Payment Format",
            "Is Laundering",
            "Pattern",
            "Pattern Details",
            "Same Group",
        ],
    )

    return df_1, df_2

In [16]:
with open("../data/HI-Small_Patterns.txt", "r") as file:
    # Read the entire content of the file
    text = file.read()

laundering_grp_rows, all_rows = parse_text(text)
df_1, df_2 = convert_to_dataframe(laundering_grp_rows, all_rows)
df_1

,Pattern,Pattern Details,Same Group
0,FAN-OUT,Max 16-degree Fan-Out,1
1,CYCLE,Max 10 hops,2
2,GATHER-SCATTER,Max 3-degree Fan-In,3
3,GATHER-SCATTER,None,4
4,GATHER-SCATTER,Max 13-degree Fan-In,5
...,...,...,...
365,GATHER-SCATTER,None,366
366,CYCLE,Max 3 hops,367
367,FAN-IN,Max 12-degree Fan-In,368
368,RANDOM,Max 1 hops,369


In [17]:
#count Patterns for df_1
df_1['Pattern'].value_counts()

Pattern
CYCLE             82
FAN-OUT           80
GATHER-SCATTER    77
RANDOM            70
FAN-IN            61
Name: count, dtype: int64

In [35]:
#take a random sample of 40 pattern groups
random.seed(40)
sample_df_1 = df_1.sample(40)
sample_df_2 = df_2[df_2["Same Group"].isin(sample_df_1["Same Group"])]
sample_df_1["Pattern"].value_counts()

Pattern
GATHER-SCATTER    9
RANDOM            8
FAN-IN            8
CYCLE             8
FAN-OUT           7
Name: count, dtype: int64

In [36]:
#export sample_df_2 to csv
sample_df_2.to_csv("../data/HI-Small_Sampled_Laundering_Patterns.csv", index=False)
sample_df_2

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Pattern,Pattern Details,Same Group
26,2022/09/01 00:04,0119,811C597B0,0048309,811C599A0,34254.65,Saudi Riyal,34254.65,Saudi Riyal,ACH,1,GATHER-SCATTER,Max 3-degree Fan-In,3
27,2022/09/01 19:27,0150240,812D22980,0048309,811C599A0,5971.98,Saudi Riyal,5971.98,Saudi Riyal,ACH,1,GATHER-SCATTER,Max 3-degree Fan-In,3
28,2022/09/04 05:06,0222,811B83280,0048309,811C599A0,50445.58,Saudi Riyal,50445.58,Saudi Riyal,ACH,1,GATHER-SCATTER,Max 3-degree Fan-In,3
29,2022/09/04 05:03,0048309,811C599A0,0048309,811C599A0,48649.42,Saudi Riyal,48649.42,Saudi Riyal,ACH,1,GATHER-SCATTER,Max 3-degree Fan-In,3
30,2022/09/04 14:59,0048309,811C599A0,0119,811C597B0,64379.45,Saudi Riyal,64379.45,Saudi Riyal,ACH,1,GATHER-SCATTER,Max 3-degree Fan-In,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,2022/09/14 13:19,020,8001B94D0,022,8000FC360,8667.21,Euro,8667.21,Euro,ACH,1,FAN-IN,Max 8-degree Fan-In,352
3110,2022/09/12 14:24,0249176,812A70ED0,0049508,812A70E80,47102.53,Saudi Riyal,47102.53,Saudi Riyal,ACH,1,CYCLE,None,358
3181,2022/09/12 15:41,023691,8021353D0,015231,80266F880,4409.63,Euro,4409.63,Euro,ACH,1,GATHER-SCATTER,None,363
3182,2022/09/14 01:58,015231,80266F880,023691,8021353D0,4409.56,Euro,4409.56,Euro,ACH,1,GATHER-SCATTER,None,363


In [37]:
#export sample_df_1 to csv
sample_df_1.to_csv("../data/HI-Small_Sampled_Laundering_Patterns_Groups.csv", index=False)

In [ ]:
#command line
#python format_kaggle_files.py ../data/HI-Small_Sampled_Laundering_Patterns.csv ../data/HI-Small_Sampled_Formatted_Laundering_Patterns.csv

In [38]:
#load csv for HI-Small_Balanced_Formatted
original_df = pd.read_csv("../data/HI-Small_Balanced_Formatted.csv")
sample_df = pd.read_csv("../data/HI-Small_Sampled_Formatted_Laundering_Patterns.csv")

original_df


,EdgeID,SourceAccountId,TargetAccountId,Timestamp,Amount Sent,Sent Currency,Amount Received,Receiving Currency,Payment Format,Is Laundering
0,748,1354,1354,10,1134.43,8,1134.43,8,5,0
1,1536,2760,2761,10,2786.09,1,2786.09,1,2,0
2,1621,2908,2908,10,16.19,0,16.19,0,5,0
3,2268,4052,4052,10,35296.17,1,35296.17,1,5,0
4,5069,8887,8887,10,14.14,4,14.14,4,5,0
...,...,...,...,...,...,...,...,...,...,...
10349,10249,15096,15121,1504510,2391.92,9,2391.92,9,0,1
10350,10053,14861,14879,1504930,3749.14,1,3749.14,1,0,1
10351,10054,14861,14880,1509490,1785.27,0,1785.27,0,0,1
10352,10055,14861,14881,1515490,2154.54,1,2154.54,1,0,1


In [39]:
sample_df

,EdgeID,SourceAccountId,TargetAccountId,Timestamp,Amount Sent,Sent Currency,Amount Received,Receiving Currency,Payment Format,Is Laundering
0,0,0,1,250,34254.65,0,34254.65,0,0,1
1,5,2,1,24430,27247.23,0,27247.23,0,0,1
2,1,2,1,70030,5971.98,0,5971.98,0,0,1
3,10,6,7,115210,9270.55,1,9270.55,1,0,1
4,6,1,0,130270,25324.89,0,25324.89,0,0,1
...,...,...,...,...,...,...,...,...,...,...
309,263,267,288,1267090,15972.49,2,15972.49,2,0,1
310,264,267,289,1269910,2242.24,11,2242.24,11,0,1
311,265,267,290,1296070,18670.02,2,18670.02,2,0,1
312,266,267,291,1305550,9625.10,2,9625.10,2,0,1


In [40]:
original_df

,EdgeID,SourceAccountId,TargetAccountId,Timestamp,Amount Sent,Sent Currency,Amount Received,Receiving Currency,Payment Format,Is Laundering
0,748,1354,1354,10,1134.43,8,1134.43,8,5,0
1,1536,2760,2761,10,2786.09,1,2786.09,1,2,0
2,1621,2908,2908,10,16.19,0,16.19,0,5,0
3,2268,4052,4052,10,35296.17,1,35296.17,1,5,0
4,5069,8887,8887,10,14.14,4,14.14,4,5,0
...,...,...,...,...,...,...,...,...,...,...
10349,10249,15096,15121,1504510,2391.92,9,2391.92,9,0,1
10350,10053,14861,14879,1504930,3749.14,1,3749.14,1,0,1
10351,10054,14861,14880,1509490,1785.27,0,1785.27,0,0,1
10352,10055,14861,14881,1515490,2154.54,1,2154.54,1,0,1


In [45]:
#count Is Laundering
original_df["Is Laundering"].value_counts()


Is Laundering
0    5177
1    5177
Name: count, dtype: int64

In [51]:
# Filter original_df to keep rows where 'Is Laundering' is 0
original_df_filtered = original_df[original_df['Is Laundering'] == 0]

#combine with sample_df
combined_df = pd.concat([original_df_filtered, sample_df], ignore_index=True)
#sort by timestamp
combined_df = combined_df.sort_values(by='Timestamp')

In [52]:
combined_df

,EdgeID,SourceAccountId,TargetAccountId,Timestamp,Amount Sent,Sent Currency,Amount Received,Receiving Currency,Payment Format,Is Laundering
0,748,1354,1354,10,1134.43,8,1134.43,8,5,0
1,1536,2760,2761,10,2786.09,1,2786.09,1,2,0
2,1621,2908,2908,10,16.19,0,16.19,0,5,0
3,2268,4052,4052,10,35296.17,1,35296.17,1,5,0
4,5069,8887,8887,10,14.14,4,14.14,4,5,0
...,...,...,...,...,...,...,...,...,...,...
5486,263,267,288,1267090,15972.49,2,15972.49,2,0,1
5487,264,267,289,1269910,2242.24,11,2242.24,11,0,1
5488,265,267,290,1296070,18670.02,2,18670.02,2,0,1
5489,266,267,291,1305550,9625.10,2,9625.10,2,0,1


In [53]:
#see spread of Timestamp where Is Laundering = 1
combined_df[combined_df["Is Laundering"] == 1]["Timestamp"].describe()

count    3.140000e+02
mean     6.390710e+05
std      2.618758e+05
min      2.500000e+02
25%      4.729000e+05
50%      6.246700e+05
75%      8.312350e+05
max      1.317610e+06
Name: Timestamp, dtype: float64

In [54]:
#save to csv
combined_df.to_csv("../data/HI-Small_Sampled_5491.csv", index=False)